<a href="https://colab.research.google.com/github/hasiburrahman1/NLP/blob/master/train_test_Embedding_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import tensorflow as tf

In [ ]:
import os
import csv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
tf.__version__ 

'2.2.0'

In [ ]:
data_train = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/train.tsv",sep='\t',encoding = "ISO-8859-1")
data_test = pd.read_csv(r"/content/drive/My Drive/Colab Notebooks/test.tsv",sep='\t',encoding = "ISO-8859-1")

In [ ]:
data_train.head()

,label,question,answer_candidate
0,0,How many genes does the human hoxD cluster con...,"In comparison , HOX genes ( HOXA3 , HOXD9 , HO..."
1,0,How many genes does the human hoxD cluster con...,"We report here a novel NUP98 partner gene , HO..."
2,0,How many genes does the human hoxD cluster con...,"Heterozygous mutations of human HOXD13 , encod..."
3,0,How many genes does the human hoxD cluster con...,The apparently balanced de novo translocation ...
4,0,How many genes does the human hoxD cluster con...,Gene-gene interactions were identified between...


In [ ]:
print("Number of rows in Train data =",data_train.shape[0])
print("Number of columns in Train data =",data_train.shape[1])

Number of rows in Train data = 44933
Number of columns in Train data = 3


In [ ]:
print("Number of rows in Test data =",data_test.shape[0])
print("Number of columns in Test data =",data_test.shape[1])

Number of rows in Test data = 9064
Number of columns in Test data = 3


##### Preprocessing for train data

In [ ]:
# Removing punctuations
data=data_train.iloc[:,1:3]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)

# Renaming column names for ease of access
list1= [i for i in range(2)]
new_Index=[str(i) for i in list1]
data.columns= new_Index

In [ ]:
' '.join(str(x) for x in data.iloc[0,0:3])

'How many genes does the human hoxD cluster contain  In comparison   HOX genes   HOXA    HOXD    HOXD     HOXD     and HOXD AS    a HOXD cluster antisense RNA       and WNT B expression were also significantly higher in sigmoid colon compared with the rectum  '

In [ ]:
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower()   

In [ ]:
' '.join(str(x) for x in data.iloc[0,0:3])

'how many genes does the human hoxd cluster contain  in comparison   hox genes   hoxa    hoxd    hoxd     hoxd     and hoxd as    a hoxd cluster antisense rna       and wnt b expression were also significantly higher in sigmoid colon compared with the rectum  '

In [ ]:
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:3]))

In [ ]:
corpus[0]

'how many genes does the human hoxd cluster contain  in comparison   hox genes   hoxa    hoxd    hoxd     hoxd     and hoxd as    a hoxd cluster antisense rna       and wnt b expression were also significantly higher in sigmoid colon compared with the rectum  '

In [ ]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Bidirectional
from tensorflow.keras.layers import Dropout

#### One hot representation

In [ ]:
### Vocabulary size
voc_size=10000

In [ ]:
onehot_repr=[one_hot(words,voc_size)for words in corpus] 



```
# This is formatted as code
```

#### Embedding Representation

In [ ]:
sent_length = len(max(corpus, key=len))
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 5643  295 1585]
 [   0    0    0 ... 2422 7781 5739]
 [   0    0    0 ... 5643 5057 1665]
 ...
 [   0    0    0 ... 2506 1954 9214]
 [   0    0    0 ...  968 2599 7278]
 [   0    0    0 ... 6208 1954 9214]]


In [ ]:
embedded_docs[0]

array([   0,    0,    0, ..., 5643,  295, 1585], dtype=int32)

# tarin dataset split into X_train and y_train





In [ ]:
# tarin dataset split into X_train and y_train
X = np.array(embedded_docs)
X_train = X

y = pd.get_dummies(data_train['label'])
y = np.array(y.iloc[:,1].values)  
y_train = y

# test dataset split into X_test and y_test


In [ ]:
# test dataset split into X_test and y_test

# Removing punctuations for test data
data = data_test.iloc[:,1:3]
data.replace("[^a-zA-Z]"," ",regex=True, inplace=True)
# Renaming column names for ease of access
list1 = [i for i in range(2)]
new_Index=[str(i) for i in list1]
data.columns= new_Index
# Convertng headlines to lower case
for index in new_Index:
    data[index]=data[index].str.lower() 
#append corpus list to all row
corpus = []
for row in range(0,len(data.index)):
    corpus.append(' '.join(str(x) for x in data.iloc[row,0:3]))
### Vocabulary size
voc_size=5000
#onehot representation
onehot_repr=[one_hot(words,voc_size)for words in corpus] 
#### Embedding Representation
sent_length = len(max(corpus, key=len))
embedded_docs=pad_sequences(onehot_repr,padding='pre',maxlen=sent_length)
X_test = np.array(embedded_docs)


y=pd.get_dummies(data_test['label'])
y_test = y.iloc[:,1].values

In [ ]:
X_train.shape

(44933, 4166)

In [ ]:
X_test.shape

(9064, 2619)

In [ ]:
y_train.shape

(44933,)

In [ ]:
y_test.shape

(9064,)

train test split

In [ ]:
'''
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)
'''

'\nfrom sklearn.model_selection import train_test_split\nX_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.33, random_state = 42)\n'

#### Implement BAG OF WORDS

In [ ]:
'''
from sklearn.feature_extraction.text import CountVectorizer

countvector=CountVectorizer(ngram_range=(2,2))
X=countvector.fit_transform(corpus)
'''

'\nfrom sklearn.feature_extraction.text import CountVectorizer\n\ncountvector=CountVectorizer(ngram_range=(2,2))\nX=countvector.fit_transform(corpus)\n'

#### TFidf Vectorizer

In [ ]:
'''
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))
X=tfidf_v.fit_transform(corpus).toarray()
'''

'\nfrom sklearn.feature_extraction.text import TfidfVectorizer\n\ntfidf_v=TfidfVectorizer(max_features=10000,ngram_range=(1,3))\nX=tfidf_v.fit_transform(corpus).toarray()\n'

### LSTM model

In [ ]:
## Creating model

embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 2619, 40)          200000    
_________________________________________________________________
lstm_6 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_6 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating Bidirectional LSTM model

embedding_vector_features=40
model1=Sequential()
model1.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model1.add(Bidirectional(LSTM(100)))
model1.add(Dropout(0.3))
model1.add(Dense(1,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2619, 40)          200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


In [ ]:
## Creating LSTM with dropout ratio model

embedding_vector_features=40
model2=Sequential()
model2.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model2.add(Dropout(0.3))
model2.add(LSTM(100))
model2.add(Dropout(0.3))
model2.add(Dense(1,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model1.summary())

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_7 (Embedding)      (None, 2619, 40)          200000    
_________________________________________________________________
bidirectional_2 (Bidirection (None, 200)               112800    
_________________________________________________________________
dropout_6 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_7 (Dense)              (None, 1)                 201       
Total params: 313,001
Trainable params: 313,001
Non-trainable params: 0
_________________________________________________________________
None


##### LSTM Model Training

In [ ]:
### Finally Training
model1.fit(X_train,y_train,validation_data=(X_train,y_train),epochs=10,batch_size=64)

Epoch 1/10


InvalidArgumentError: ignored

##### Performance Metrics And Accuracy of LSTM model

In [ ]:
y_pred1= np.argmax(model1.predict(X_test), axis=-1) 

In [ ]:
## Import library to check accuracy
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,f1_score

In [ ]:
matrix=confusion_matrix(y_test,y_predict)
print(matrix)

In [ ]:
accuracy_score(y_test,y_pred1)

In [ ]:
report=classification_report(y_test,y_pred1)
print(report)

### RandomForest Classifier

In [ ]:
## implement RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier

In [ ]:
randomclassifier=RandomForestClassifier(n_estimators=200,criterion='entropy')
randomclassifier.fit(X_train,y_train.ravel())

y_predict = randomclassifier.predict(X_test)

ValueError: ignored

In [ ]:
matrix=confusion_matrix(y_test,y_predict)
print(matrix)
score=accuracy_score(y_test,y_predict)
print(score)

In [ ]:
report=classification_report(y_test,y_predict)
print(report)